# MC Control - Blackjack Environment

In [1]:
def incremental_Q(state_action_freq, state_action_Q, total_return): 
    
    
    temp_diff = total_return - state_action_Q               #state_action_return
    updated_Q = state_action_Q + (1/state_action_freq) * (temp_diff)
    

    return updated_Q

In [ ]:
def mc_control():     
    
    sa_rewards = {}
    sa_Q= {}
    sa_optimQ = {}
    num_state_val_pair = {}
    s_ar = {}
    Q = 0.5
    
    eplison = 0.5
    
    for i in range(num_eps):
        
        #generate new dealer and player every game. so, generate an episode.
    
        is_terminal = False # means that game is NOT over. 
        
        
        while not is_terminal: 

            action = random.randint(0,1) # epsilon policy
            
            n_state, reward, is_terminal, info = env.step(action)
            #print((state, action))
            episode_sa.append((state, action))
            rewards_sa.append(reward)
            if state not in s_ar.keys():
                s_ar[state] = []
            s_ar[state].append((action, reward))
            
                
            
        is_terminal = False
        
        #first occurrence logic
        visited_sa = []
        for i in range(len(episode_sa)): 
            
            if episode_sa[i] not in visited_sa: 
                
                # accumulate award after first occurrence. 
                sa_rewards[episode_sa[i]] = []
                sa_rewards[episode_sa[i]].append(sum(reward_sa[i:]))
                
                # start the count for the number of times the first occurence shows up
                num_state_value_pair[episode_sa[i]] = 1
                
                # fresh Q value and dict list value
                
                #sa_Q[episode_sa[i]] = [0]
                
                # add to visited list
                visited_sa.append(episode_sa[i])
            else:
                 num_state_value_pair[episode_sa[i]] += 1
                
           #is this right?
            
            Q = incremental_Q(num_state_value_pair[episode_sa[i]], Q, sa_rewards[episode_sa[i]])
            sa_Q[episode_sa[i]].append(Q)
            
        #sooo i also need to keep track of the number of the distinct states and correspond those to their actions, 
        #and then correspond those to their Q values. Ugh.
        
        # e-greedy!!
         
        for state in s_ar: 
            
            
            
            
            
        

# Following Tutorial

In [2]:
import numpy as np
import gym
from collections import defaultdict
import sys
import json
#from plot_utils import plot_policy, plot_win_rate

In [5]:
env = gym.make('Blackjack-v0')
state = env.reset()
action_space = env.action_space.n

print(state)
print(action_space)

(6, 6, False)
2


State: First value is the sum of the player's first two cards, Second value is the Dealer Card that is showing, third value is if there is a usable ace or not. There is a usable ace when you draw an ace that can be used as an "11" to win the game



Action Space: You get 2 actions, hit or stick

The Q function maps states (which are three tuples) to two actions

In [6]:
Q = defaultdict(lambda: np.zeros(2))

# how does this work? 

Q[env.reset()]

array([0., 0.])

index 0 is the Q value for getting a hit, index 1 is the Q value for getting a stick

In [22]:
# hyperparameters

num_episodes = 1000000
epsilon = 1
epsilon_min = 0.05
gamma = 1
alpha = 0.03

In [42]:
def monte_carlo_control(num_episodes, epsilon, epsilon_min, alpha, gamma = 1): 
    
    Q = defaultdict(lambda: np.zeros(2))
    
    rewards_all_episodes = [] # for calculating the winrate
    
    for i in range(1, num_episodes + 1):
        
        #update epsilon values. this is how the epsilon value decays
        epsilon = max(epsilon_min, epsilon * 0.999999) # in every iteration of this for loop, epsilon decays by 1-0.999999 percent
        #so, after many episodes, this epsilon value is going to get so low (closer to epsilon_min), that we be explorting much more than er
        #explore. however, epsilon_min is there so we are still guaranteed some degree of exploration
        
        if i % 1000 == 0: 
            print("\rEpisode {}/{}.".format(i, num_episodes), end = "")
            sys.stdout.flush()
        
        #need to write function to generate episodes
            #action regards to the epsilon greedy policy (also need function for epsilon_greedy)
            # also return states, actions, and rewards
        experience = generate_episode(Q, epsilon) #one game
        
        states, actions, rewards = zip(*experience) # give us states actions and rewards in separate lists
        rewards = np.array(rewards)
        rewards_all_episodes.append(sum(rewards))
        
       
        
        #need to loop over the timesteps for that episode or experience
        
        for i, state in enumerate(states):
            discounts = [gamma ** i for i in range(len(rewards[i:]))]
            returns = sum(rewards[i:] * discounts)
            Q[state][actions[i]] += alpha * (returns -  Q[state][actions[i]])
            # alpha is a constant term that leads the loss in direction of the update value and to make the Q value
            # more accurate over time
            policy = dict((state, np.argmax(q_value)) for state, q_value in Q.items())
            
    return Q, policy, rewards_all_episodes

In [12]:
def generate_episode(Q, epsilon):
    
   
    state = env.reset() # everytime we generate an episode, we need to do this so that we can intialize a random state 
    # as the games starting point
    episode = [] # need to keep track of the states, actions, and rewards from the episode (represent as a tuple)
    action =  epsilon_greedy_policy(Q, state, epsilon) #defined with epsilon greedy policy
    
    while True: # keep running until the game ends (while done is false)
    
        next_state, reward, done, info = env.step(action)
        state = next_state # the current state is going to turn into the next state, and so on. did not have this in my original function
        episode.append((state, action, reward)) # i had this a dictionary with s,a as the key and the reward as the dictionary
        
        if done == True : #why do we need this?
            break
        
    return episode
    

In [14]:
def epsilon_greedy_policy(Q, state, epsilon):
    
    # in this policy, we take the random action epsilon * 100 percent of the time. The rest of the time (1-e) we are going to 
    #choose the action that maximized the Q value. This is the argmax of the Q function for some state. Select the action that has the highest value
    
        probs = np.zeros(2)
        optimal_action = np.argmax(Q[state]) # will either be a zero or a 1
        sub_optimal_action = np.abs(optimal_action - 1)# the OTHER action, (lower Q value). if optimal value is 0, sub is 1, and vice verse
        
        
        probs[optimal_action] = (1 - epsilon) + (epsilon/2)
        probs[sub_optimal_action] = (epsilon/2)
        
        # choose an action based on the probabilities assigned
        
        action = np.random.choice(np.arange(2), p = probs) # choose the choices based on the probabilities and the indexes
        # for the optimal and sub action
    
        # how do we know the other in which the optimal and sub will be in probs?
        return action
    

In [16]:
generate_episode(Q, 0.7)

[((6, 5, False), 1, 0.0),
 ((12, 5, False), 1, 0.0),
 ((14, 5, False), 1, 0.0),
 ((18, 5, False), 1, 0.0),
 ((21, 5, False), 1, 0.0),
 ((31, 5, False), 1, -1.0)]

reward is going to be zero when the episode hasn't terminated yet

In [47]:
Q, policy, rewards_all_episodes = monte_carlo_control(num_episodes, epsilon, epsilon_min, alpha, gamma)

Episode 14000/1000000.

KeyboardInterrupt: 

In [48]:
print(len(Q))

1
